# Automated ML
TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.automl
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset
from azureml.train.automl.utilities import get_primary_metrics
from azureml.train.automl import AutoMLConfig
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.automl.core.shared import constants


import os
import json
import joblib
import logging
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

## Initializing a Workspace

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-190098
aml-quickstarts-190098
southcentralus
1b944a9b-fdae-4f97-aeb1-b7eea0beac53


TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

## Creating an AML Experiment

In [3]:
# choose a name for experiment
experiment_name = 'automl-heart-failure'
experiment = Experiment(ws, experiment_name)
run = experiment.start_logging()

experiment

Name,Workspace,Report Page,Docs Page
automl-heart-failure,quick-starts-ws-190098,Link to Azure Machine Learning studio,Link to Documentation


## Creating a Compute Cluster

In [4]:
amlcompute_cluster_name = "hf-compute"

try:
    compute_target = ComputeTarget(workspace = ws, name = amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_DS3_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded......................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Data Wrangling

## Data Gathering

### Overview
In this project, the Heart Failure Prediction dataset from Kaggle is used. The description of the dataset is provided below.

**The dataset has the following features:**  
* age: Age (numeric)
* anaemia: Decrease of red blood cells or hemoglobin (boolean)
* creatinine_phosphokinase: Level of the CPK enzyme in the blood (mcg/L)
* diabetes: If the patient has diabetes (boolean)
* ejection_fraction: Percentage of blood leaving the heart at each contraction (percentage)
* high_blood_pressure: If the patient has hypertension (boolean)
* platelets: Platelets in the blood (kiloplatelets/mL)
* serum_creatinine: Level of serum creatinine in the blood (mg/dL)
* serum_sodium: Level of serum sodium in the blood (mEq/L)
* sex: Woman or man (binary)

**The task:**  
Developing a ML model to predict death events using 12 clinical features.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
### Gathering the Dataset
data_path = "https://raw.githubusercontent.com/ugururesin/Azure-ML-Capstone-Project/main/dataset/heart_failure_clinical_records_dataset.csv"


exist = False
key = "heart-failure"
description_text = "heart failure dataset for binary classification"

if key in ws.datasets.keys(): 
        exist = True
        dataset = ws.datasets[key] 
        

if not exist:
        #Creating automl dataset and registering it into the workspace
        dataset = Dataset.Tabular.from_delimited_files(data_path)        
        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

## Data Assesing

In [6]:
df = dataset.to_pandas_dataframe()
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [7]:
#Creating training and testing datasets
x = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

train_df = pd.concat([x_train, y_train], axis=1)
test_df = pd.concat([x_test, y_test], axis=1)

train_df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
52,60.0,0,3964,1,62,0,263358.03,6.8,146,0,0,43,1
270,44.0,0,582,1,30,1,263358.03,1.6,130,1,1,244,0
260,55.0,0,66,0,40,0,203000.00,1.0,138,1,0,233,0
125,43.0,1,358,0,50,0,237000.00,1.3,135,0,0,97,0
15,82.0,1,379,0,50,0,47000.00,1.3,136,1,0,13,1


In [8]:
#Since pandas dataframe is not supported, the dataset needs to be converted to tabular dataset
if not os.path.isdir('dataset'):
    os.mkdir('dataset')

pd.DataFrame(train_df).to_csv("dataset/train_data.csv", index=False)
pd.DataFrame(test_df).to_csv("dataset/test_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir = './dataset', target_path = 'heart-failure-dataset/train_data.csv')

train_data = Dataset.Tabular.from_delimited_files(path = ds.path('heart-failure-dataset/train_data.csv'))

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


In [9]:
train_data

{
  "source": [
    "('workspaceblobstore', 'heart-failure-dataset/train_data.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

### Checking the target feature

In [10]:
df['DEATH_EVENT'].value_counts()

0    203
1     96
Name: DEATH_EVENT, dtype: int64

In [11]:
get_primary_metrics("classification")

['AUC_weighted',
 'norm_macro_recall',
 'precision_score_weighted',
 'accuracy',
 'average_precision_score_weighted']

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

* **experiment_timeout_minutes**: 15 minutes is set since the dataset is small.

* **iterations**: 50 is set since the dataset is small.  

* **max_concurrent_iterations**: Represents the maximum number of iterations that would be executed in parallel.

* **n_cross_validations**: To avoid overfitting and ensure randomness.

* **primary_metric**: Since the target response is imbalanced, accuracy is not a good evaluation metric. So, auc is selected.  

* **task**: Classification is selected since the response variable is binary (0 or 1).  

* **enable_early_stopping**: Activated to avoid overfitting.

In [12]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 15,
    "iterations": 50,
    "max_concurrent_iterations": 3,
    "n_cross_validations": 3,
    "primary_metric" : 'AUC_weighted',
    "featurization" : 'auto',
    "verbosity": logging.INFO
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task = "classification", 
    training_data = train_data,
    label_column_name="DEATH_EVENT",
    enable_early_stopping= True,
    debug_log = "automl_errors.log",
    compute_target = compute_target,
    **automl_settings
)
                             

In [13]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)
remote_run.wait_for_completion(show_output=True)

Submitting remote run.
No run_configuration provided, running on hf-compute with default configuration
Running on remote compute: hf-compute


Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart-failure,AutoML_bf1ec23c-5e81-4353-acba-4ed7d22c1a01,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart-failure,AutoML_bf1ec23c-5e81-4353-acba-4ed7d22c1a01,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feat

{'runId': 'AutoML_bf1ec23c-5e81-4353-acba-4ed7d22c1a01',
 'target': 'hf-compute',
 'status': 'Completed',
 'startTimeUtc': '2022-03-27T11:55:04.68859Z',
 'endTimeUtc': '2022-03-27T12:09:59.111454Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '50',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'hf-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-heart-failure","subscription_id":"1b944a9b-fdae-4f97-aeb1-b7eea0beac53","resource_group":"aml-quickstarts-190098","workspace_name":"quick-starts-ws-190098","region":"southcentralus","compute_target":"hf-compute

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
best_run, fitted_model = remote_run.get_output()

In [16]:
best_metrics = best_run.get_metrics()

print("Best run id : {}".format(best_run.id), 
      "AUC : {}".format(best_metrics['accuracy']), 
      "Best metrics : {}".format(best_metrics), 
      "Best model : {}".format(fitted_model), sep = '\n')

Best run id : AutoML_bf1ec23c-5e81-4353-acba-4ed7d22c1a01_34
AUC : 0.9599162479061977
Best metrics : {'f1_score_weighted': 0.9598505284759061, 'norm_macro_recall': 0.9056894368315808, 'average_precision_score_micro': 0.9892696970870007, 'f1_score_micro': 0.9599162479061977, 'average_precision_score_weighted': 0.9878255687168603, 'precision_score_micro': 0.9599162479061977, 'recall_score_micro': 0.9599162479061977, 'average_precision_score_macro': 0.981880481677751, 'precision_score_weighted': 0.9607936122419382, 'AUC_micro': 0.9892081445754736, 'AUC_weighted': 0.9899110970702777, 'balanced_accuracy': 0.9528447184157903, 'f1_score_macro': 0.9527035436112196, 'accuracy': 0.9599162479061977, 'recall_score_weighted': 0.9599162479061977, 'AUC_macro': 0.9899110970702777, 'log_loss': 0.17661062409477676, 'weighted_accuracy': 0.9650258704544408, 'recall_score_macro': 0.9528447184157903, 'precision_score_macro': 0.953823966463235, 'matthews_correlation': 0.9065485189384197, 'confusion_matrix': 

In [17]:
#the fitted model
_, fitted_model = remote_run.get_output()
print(fitted_model)

In [18]:
#TODO: Save the best model
best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')
joblib.dump(fitted_model, filename= "outputs/automl_model.pkl")

['outputs/automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
# downloading score.py and env file
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
# registering the model
model_name = best_run.properties['model_name']
description = 'AutoML model trained on heart failure data so as to predict whether death event occurs based on the clinical values'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)

print(remote_run.model_id) # This will be written to the script file later in the notebook.

AutoMLbf1ec23c534


In [22]:
#Donwloading the scoring file
environment_name = 'heart-failure-ws'
environment = Environment.from_conda_specification(name=environment_name, file_path='conda_dependencies.yml')
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

#Inference configuration
inference_config = InferenceConfig(entry_script = 'score.py', environment = environment)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(
    workspace = ws,
    name='heart-failure-ws',
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True)

service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-03-27 12:46:56+00:00 Creating Container Registry if not exists.
2022-03-27 12:46:56+00:00 Registering the environment.
2022-03-27 12:46:57+00:00 Use the existing image.
2022-03-27 12:46:58+00:00 Generating deployment configuration.
2022-03-27 12:46:58+00:00 Submitting deployment to compute.
2022-03-27 12:47:05+00:00 Checking the status of deployment heart-failure-ws..
2022-03-27 12:50:18+00:00 Checking the status of inference endpoint heart-failure-ws.
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 0511d211-1f83-46e2-b7d8-d8a32c5551cd
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: heart-failure-ws. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image d96ceb558630468fa1d411d4099cddc2.azurecr.io/azureml/azureml_458ecd79cd67b35abd40504d66eb2a07 locally. Please refer to h

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 0511d211-1f83-46e2-b7d8-d8a32c5551cd
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: heart-failure-ws. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image d96ceb558630468fa1d411d4099cddc2.azurecr.io/azureml/azureml_458ecd79cd67b35abd40504d66eb2a07 locally. Please

### Consume the Model

In [ ]:
# creating a randomly selected test data
test_data = df.sample(10)
test_label = test_data.pop('DEATH_EVENT')

# converting the records to a json data file
import json
data_json = test_data.to_dict(orient='records')
data = json.dumps({'data': data_json})

In [ ]:
# getting inference
output = webservice.run(data)
print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
webservice.get_logs()

In [ ]:
#webservice.delete()
#compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
